In [5]:
import pandas as pd
from math import sqrt, fabs, exp
import matplotlib.pyplot as plot
from sklearn.linear_model import enet_path
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sklearn import ensemble
import numpy as np

In [14]:
dfX = pd.read_csv("X_train.csv")
dfTest = pd.read_csv("X_test.csv")
dfY = pd.read_csv("y_train.csv")
XPred = dfTest.iloc[:, 1:].values
X = dfX.iloc[:, 1:].values
y = dfY.iloc[:, 1:].values

#Check there are no nans
print((X == np.nan).astype(int).sum()) 
print((y == np.nan).astype(int).sum()) 

0
0


### Drawing 30% test sample may not preserve population proportions

___

##### Solution is to sample each population of a specific label independently #####

In [11]:
#stratified sampling by labels
nrows = len(X)
classes = [1, 2]

# sample only the zero-labeled points (then sample the remaning data points in the next for loop)
xTemp = [X[i] for i in range(nrows) if y[i] == 0]
yTemp = [y[i] for i in range(nrows) if y[i] == 0]
xTrain, xTest, yTrain, yTest = train_test_split(xTemp, yTemp, test_size=0.30, random_state=531)

# append the other data points from the other classes
for iLabel in classes:
    xTemp = [X[i] for i in range(nrows) if y[i] == iLabel]
    yTemp = [y[i] for i in range(nrows) if y[i] == iLabel]
    
    #form train and test sets on segregated subset of examples    
    xTrainTemp, xTestTemp, yTrainTemp, yTestTemp = train_test_split(xTemp, yTemp, test_size=0.30, random_state=531) 
    
    #accumulate    
    xTrain = numpy.append(xTrain, xTrainTemp, axis=0)    
    xTest = numpy.append(xTest, xTestTemp, axis=0)    
    yTrain = numpy.append(yTrain, yTrainTemp, axis=0)    
    yTest = numpy.append(yTest, yTestTemp, axis=0)

## Search for optimal hyperparameters for GBoost

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import DataConversionWarning
import warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [ ]:
nTreeList = np.arange(50, 1700, 35)
max_feats = np.arange(3, 800, 25)
depths = np.arange(2, 11, 1)
learnRate = 0.05
subSamp = 1
search_model_params = GridSearchCV(
    ensemble.GradientBoostingClassifier(learning_rate=learnRate, subsample=subSamp),{
    'n_estimators': nTreeList,
    'max_depth':depths,
    'max_features': max_feats   
    }, cv=5, return_train_score = False
)
search_model_params.fit(X, y)
print(search_model_params.best_params_)
print(search_model_params.best_score_)
predictions = search_model_params.predict(XPred)

### Save predictions

In [ ]:
dfPredictions = pd.DataFrame(predictions)
dfPredictions.index.name = "id"
df.to_csv("gboostedtask2noimbalance.csv", header = ['y'], index=True)